# Projeto 1 - Ciência dos Dados

Nome: Leonardo Almodi dos Reis

Nome: Theo Konigsberger Ferros

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\theok\OneDrive\Área de Trabalho\Projeto 1 Cdados 2023-2\Proj1_LEOTHEO_CDADOS


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Target
0,Os livros e suas capas estão ótimos. Bonitos e...,0
1,Como pode uma editora aproveitar o sucesso do ...,2
2,"Muito ruim, uma série de repetições desnecessá...",2
3,Cansativo. O kindle não facilita as idas e vin...,2
4,O final decepcionante da trilogia me fez arrep...,2


In [4]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,Livros e caixa vieram danificados. Parte exter...,2
1,"A ""obra"" é uma fraude completa. O autor é um a...",2
2,Trata de modo superficial o tema da velocidade...,2
3,"Descreve um relacionamento abusivo, que é bem ...",2
4,"O box chegou com amassados, riscos e com uma e...",2


0 = Neutro;
1 = Postivos;
2 = Negativo.

___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

O assunto do projeto é sobre Reviews de consumidores sobre livros e suas entregas, e autores, em algum site (AMAZON). O que foi usado como relevante ou nao para a construcao do target foram se os reviews estavam escritos, de modo geral, em afirmacoes positivas, ou negativas, ou caso houvesse um balanco, neutras.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
def cleanup(text):

    # Função de limpeza muito simples que troca alguns sinais básicos por espaços
    
    punctuation = '[´"!-.:?;$''/\|@,><%&*()_+^~`}{+=#*n]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

 

In [6]:
for i in range(len(train.Mensagem)-1):
    train.Mensagem[i] = cleanup(train.Mensagem[i])

C:\Users\theok\AppData\Local\Temp\ipykernel_146028\880326560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Mensagem[i] = cleanup(train.Mensagem[i])


In [7]:
lista_total = []
lista_neutro = []
lista_positivos = []
lista_negativos = []

for i in range(len (train.Mensagem)-1):
    # Separando os rows do DataFrame
    x = train.Mensagem[i].lower().split()
    

    t = 0
    while t < len(x):
        lista_total.append(x[t])
        t+=1

    if train.iloc[i,1] == 0:
        n = 0
        while n < len(x):
            lista_neutro.append(x[n])
            n+=1

    elif train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_positivos.append(x[j])
            j+=1
        
    elif train.iloc[i,1] == 2:
        k = 0
        while k < len(x):
            lista_negativos.append(x[k])
            k+=1



In [8]:
serie_total = pd.Series(lista_total)
serie_neutro = pd.Series(lista_neutro)
serie_positivos = pd.Series(lista_positivos)
serie_negativos = pd.Series(lista_negativos)


In [9]:
abs_total = serie_total.value_counts()
abs_neutro = serie_neutro.value_counts()
abs_positivos = serie_positivos.value_counts()
abs_negativos = serie_negativos.value_counts()

rel_total = serie_total.value_counts(True)
rel_neutro = serie_neutro.value_counts(True)
rel_positivos = serie_positivos.value_counts(True)
rel_negativos = serie_negativos.value_counts(True)


In [10]:
P_neutro = len(abs_neutro)/len(abs_total)
P_positivo = len(abs_positivos)/len(abs_total)
P_negativo = len(abs_negativos)/len(abs_total)

lista_classificacao = []

for m in train.Mensagem:
    
    lista_palavras = m.split()

    P_positivo = 1
    P_negativo = 1
    P_neutro = 1

    for p in lista_palavras:
            if p in abs_neutro:
                P_neutro *= (abs_neutro[p]+1)/(len(abs_neutro)+ len(abs_total))
            else:
                P_neutro *= 1/(len(abs_neutro) + len(abs_total))

            if p in abs_positivos:
                P_positivo *= (abs_positivos[p]+1)/(len(abs_positivos) + len(abs_total))
            else:
                P_positivo  *= 1/(len(abs_positivos) + len(abs_total))

            if p in abs_negativos:
                P_negativo *= (abs_negativos[p]+1)/(len(abs_negativos) + len(abs_total))
            else:
                P_negativo *= 1/(len(abs_negativos) + len(abs_total))


    if P_positivo > P_negativo and P_positivo > P_neutro:
        lista_classificacao.append(1)
    elif P_negativo > P_positivo and P_negativo > P_neutro:
        lista_classificacao.append(2)
    else:
        lista_classificacao.append(0)

train['Target (código)'] = lista_classificacao
train

,Mensagem,Target,Target (código)
0,Os livros e suas capas estão ótimos Boitos e s...,0,2
1,Como pode uma editora aproveitar o sucesso do ...,2,2
2,Muito ruim uma série de repetições desecessári...,2,2
3,Casativo O kidle ão facilita as idas e vidas o...,2,2
4,O fial decepcioate da trilogia me fez arrepede...,2,2
...,...,...,...
295,O livro é excelete porém chegou todo amassado ...,1,2
296,O livro é de autoajuda e deveria ser vedido co...,2,2
297,A história em si é boa mas é bem casativa Tete...,0,2
298,Estava tetado há dias cotiuar ledo sem que fic...,2,2


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [11]:
for i in range(len(test.Mensagem)-1):
    test.Mensagem[i] = cleanup(test.Mensagem[i])

C:\Users\theok\AppData\Local\Temp\ipykernel_146028\1958566861.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Mensagem[i] = cleanup(test.Mensagem[i])


In [12]:
total = []
neutro = []
positivos = []
negativos = []

for i in range(len (test.Mensagem)-1):
    # Separando os rows do DataFrame
    x = test.Mensagem[i].lower().split()
    

    t = 0
    while t < len(x):
        total.append(x[t])
        t+=1

    if test.iloc[i,1] == 0:
        n = 0
        while n < len(x):
            neutro.append(x[n])
            n+=1

    elif test.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            positivos.append(x[j])
            j+=1
        
    elif test.iloc[i,1] == 2:
        k = 0
        while k < len(x):
            negativos.append(x[k])
            k+=1


In [13]:
serie_total_test = pd.Series(lista_total)
serie_neutro_test = pd.Series(lista_neutro)
serie_positivos_test = pd.Series(lista_positivos)
serie_negativos_test = pd.Series(lista_negativos)

In [14]:
abs_total_test = serie_total.value_counts()
abs_neutro_test = serie_neutro.value_counts()
abs_positivos_test = serie_positivos.value_counts()
abs_negativos_test = serie_negativos.value_counts()

rel_total_test = serie_total.value_counts(True)
rel_neutro_test = serie_neutro.value_counts(True)
rel_positivos_test = serie_positivos.value_counts(True)
rel_negativos_test = serie_negativos.value_counts(True)

In [16]:
P_test_neutro = len(abs_neutro_test)/len(abs_total_test)
P_test_positivo = len(abs_positivos_test)/len(abs_total_test)
P_test_negativo = len(abs_negativos_test)/len(abs_total_test)

lista_test_classificacao = []

for m in test.Mensagem:
    
    lista__test_palavras = m.split()

    P_test_positivo = 1
    P_test_negativo = 1
    P_test_neutro = 1

    for p in lista_palavras:
            if p in abs_neutro_test:
                P_test_neutro *= (abs_neutro_test[p]+1)/(len(abs_neutro_test)+ len(abs_total_test))
            else:
                P_test_neutro *= 1/(len(abs_neutro_test) + len(abs_total_test))

            if p in abs_positivos_test:
                P_test_positivo *= (abs_positivos_test[p]+1)/(len(abs_positivos_test) + len(abs_total_test))
            else:
                P_test_positivo  *= 1/(len(abs_positivos_test) + len(abs_total_test))

            if p in abs_negativos_test:
                P_test_negativo *= (abs_negativos_test[p]+1)/(len(abs_negativos_test) + len(abs_total_test))
            else:
                P_test_negativo *= 1/(len(abs_negativos_test) + len(abs_total_test))


    if P_test_positivo > P_test_negativo and P_test_positivo > P_test_neutro:
        lista_test_classificacao.append(1)
    elif P_test_negativo > P_test_positivo and P_test_negativo > P_test_neutro:
        lista_test_classificacao.append(2)
    else:
        lista_test_classificacao.append(0)

test['Target (código)'] = lista_test_classificacao
test.head(5)

,Mensagem,Target,Target (código)
0,Livros e caixa vieram daificados Parte extera ...,2,2
1,A obra é uma fraude completa O autor é um astr...,2,2
2,Trata de modo superficial o tema da velocidade...,2,2
3,Descreve um relacioameto abusivo que é bem dif...,2,2
4,O box chegou com amassados riscos e com uma es...,2,2


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**